In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
from PIL import Image
import sys
import time

# Make sure that caffe is on the python path:
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples/hed/
sys.path.insert(0, caffe_root + 'python')

import caffe

In [2]:
color_table = np.array([  [  0,   0,   0],
                          [128,   0,   0],
                          [  0, 128,   0],
                          [128, 128,   0],
                          [  0,   0, 128],
                          [128,   0, 128],
                          [  0, 128, 128],
                          [128, 128, 128],
                          [ 64,   0,   0],
                          [192,   0,   0],
                          [ 64, 128,   0],
                          [192, 128,   0],
                          [ 64,   0, 128],
                          [192,   0, 128],
                          [ 64, 128, 128],
                          [192, 128, 128],
                          [  0,  64,   0],
                          [128,  64,   0],
                          [  0, 192,   0],
                          [128, 192,   0],
                          [  0,  64, 128] ])

data_root = '../../data/SBD/'
img_root = data_root + 'img/'

with open(data_root+'val.txt') as f:
    test_lst = f.readlines()    
test_lst = [x.strip() for x in test_lst]

In [3]:
# Visualization
def plot_single_scale(scale_lst, size):
    pylab.rcParams['figure.figsize'] = size, size/2
    
    plt.figure()
    for i in range(0, len(scale_lst)):
        s=plt.subplot(1,2,i+1)
        plt.imshow(scale_lst[i])
        s.set_xticklabels([])
        s.set_yticklabels([])
        s.yaxis.set_ticks_position('none')
        s.xaxis.set_ticks_position('none')
    plt.tight_layout()

In [4]:
# remove the following two lines if testing with cpu
caffe.set_mode_gpu()
caffe.set_device(1)
# load net
net = caffe.Net('test.prototxt', 'snapshots/pretrain_sbd_iter_4000.caffemodel', caffe.TEST)

In [5]:
start_time = time.time()
for idx in [100]:
    im = Image.open(img_root+test_lst[idx]+'.jpg')
    in_ = np.array(im, dtype=np.float32)
    in_ = in_[:,:,::-1]
    in_ -= np.array((104.00698793,116.66876762,122.67891434))
    in_ = in_.transpose((2, 0, 1))
    
    # shape for input (data blob is N x C x H x W), set data
    net.blobs['data'].reshape(1, *in_.shape)
    net.blobs['data'].data[...] = in_
    # run net and take argmax for prediction
    net.forward()
    
    # show results
    out = net.blobs['score'].data[0].argmax(axis=0)
    show = np.ndarray((out.shape[0],out.shape[1],3), dtype=np.uint8)
    show = color_table[out][:,:,::-1]
    plot_single_scale([im,show], 22)
    
diff_time = time.time() - start_time
print 'Detection took {:.3f}s per image'.format(diff_time/len(test_lst))

Detection took 0.000s per image
